In [ ]:
import perceval as pcvl
import numpy as np
from collections import Counter
pcvl.__version__

# BasicStates

In Linear Optical Circuits, photons can have many discrete degrees of freedom, called modes. 
It can be the frequency, the polarisation, the position, or all of them.

We represent these degrees of freedom with Fock states. If we have $n$ photons over $m$ modes, the Fock state $|s_1,s_2,...,s_m\rangle$ means we have $s_i$ photons in the $i^{th}$ mode. Note that $\sum_{i=1}^m s_i =n$.

In Perceval, we will use the module `pcvl.BasicState`

## Defining and manipulating BasicStates

In [ ]:
# BasicState can be constructed from a bra-ket string
s=pcvl.BasicState("|0,1>")
# or from a list
assert(s == pcvl.BasicState([0,1]))
print(s[0], s[1])

In [ ]:
print("type of s is", type(s))
# BasicState will print as a string, s.n is the number of photons, s.m the number of modes (which is also the len of the BasicState)
print(f"s='{s}', n={s.n}, m={s.m}, m={len(s)}")

<div class="alert alert-info">
Note the type is <code>exqalibur.FockState</code> and not <code>pcvl.BasicState</code>!<br>
&rarr; <code>perceval</code> library is using behind the scene a c++-optimized library <code>exqalibur</code> that is in charge of all the heavy lifting - we never use it directly, but it is where all the simulation work happens!
</div>

In [ ]:
# BasicState does not support assignment
try:
    s[0]=1
except Exception as e:
    print("ERROR:", e)

In [ ]:
# let us define a function that does that!
def assign(s, k, n_k):
    r"""assigns n_k photon in the k-th mode of s, and returns the new BasicState"""
    assert(isinstance(s, pcvl.BasicState))
    assert(n_k < s.m)
    ### ENTER CODE HERE
    ### END CODE
    return new_s

# check if it works
new_s = assign(s, 0, 1)
assert(new_s == pcvl.BasicState("|1,1>"))

### Working with annotations

Instead of simple fock states, we can all annotate each photon with an annotation. In general an annotated photon is represented by `{x:y}`
where `x` is the name of the annotation, and `y` its value - the value has to be numeric except for polarization.
A special annotation named `P` is the photon polarization and `perceval` knows about its semantic - the value can be `H` (horizontal), `V` (vertical), `D` (diagonal)...

In [ ]:
a_bs = pcvl.BasicState("|{P:H},{P:V},{P:D}>")
print(a_bs, a_bs[0],a_bs[1])

However, you can use any name you like and even combine annotations:

In [ ]:
pcvl.BasicState("|{color:0,P:H},{color:1,P:V}>")

Annotation order does not matter:

In [ ]:
assert(pcvl.BasicState("|{color:0,P:H}>")==pcvl.BasicState("|{P:H,color:0}>"))

You can check if the basic state has an annotation and obtains the annotation of the photons on a specific mode:

In [ ]:
s=pcvl.BasicState("|{P:H,color:0},0,1>")
assert(s.has_annotations)
print(s.get_mode_annotations(0))

Each annotation behaves as a dictionary of key, value:

In [ ]:
photon_idx = 0
for k in range(s.m):
    for annotation in s.get_mode_annotations(k):
        photon_annotations=[]
        for (annot_name,annot_value) in annotation:
            photon_annotations.append(annot_name)
        print(f"photon {photon_idx} in mode {k} has annotations {photon_annotations}")
        photon_idx += 1

In [ ]:
# let us define a function checking if one photon in a basic state has a polarization annotation
def has_polarization(s):
    if not s.has_annotations:
        return False
    ### ENTER CODE HERE
    ### END CODE
    return False
assert(not has_polarization(pcvl.BasicState("|0,1>")))
assert(not has_polarization(pcvl.BasicState("|{color:0}>")))
assert(has_polarization(pcvl.BasicState("|1,{color:0,P:H},{_:1}>")))

Last we can clear all annotations on a basic state:

In [ ]:
print("before:", s)
s.clear_annotations()
print("after:", s)

## Operations on BasicStates

### tensor product
`*` defines tensor product (`s1*s2` ~ $s_1\otimes s_2$)

In [ ]:
print(pcvl.BasicState([0,1])*pcvl.BasicState([2,3]))

And `**` the tensor power (`s**n` ~ $s^{\otimes n}$), typically useful to initialize an input state

In [ ]:
s2 = pcvl.BasicState([0,1])**4
print(s2)

In [ ]:
# python slice operator applies smoothly
print(s2[1:4])

# State Vectors

A state vector is defined as a general quantum state: $\sum_i \alpha_i.s_i$ where $s_i$ are `BasicState`and $\alpha_i$ their complex parameters/

StateVectors can be build directly with `+`, `-` operators can be used to create state superposition:

In [ ]:
sv1 = pcvl.BasicState([0,1])+pcvl.BasicState([1,0])

In [ ]:
# this is again an optimized `exqalibur` object!
type(sv1)

In [ ]:
# let us check the value
print(sv1)

<div class="alert alert-info">So $sv_1=\frac{\sqrt 2}{2}(\ket{1,0}+\ket{0,1})$. Note the <code>sqrt(2)/2</code> that was introduced: what even you are doing on the <code>StateVector</code>, they keep the parameters normalized so that $\sum_i |\alpha_i|^2=1$

In [ ]:
# one easy way to check this is as follows, the multiplication of a StateVector by a constant does not change its value
assert(2*pcvl.StateVector([0,1])==pcvl.StateVector([0,1]))

except when building a superposed state:

In [ ]:
sv2 = pcvl.StateVector([0,1])-2j*pcvl.StateVector([2,3])
print(sv2)

<div class="alert alert-info">
    Did you notice that it is being represented using `2*sqrt(5)/5` ($\frac{2\sqrt{5}}{5}$) and not numerical values? This is a built-in feature trying to recognize remarkable numbers when we print them - this helps the reader. This number simplification is built in the string conversion and can be disabled by addint `nsimplify=False` parameter in the implicit `__str__` function used to print the `StateVector` as follows 
</div>

In [ ]:
# switch off number conversion
print(sv2.__str__(nsimplify=False))

<div class="alert alert-warning">This is clearly less intuitive but we now see that the parameters are actually complex values!!!</div>

We can also use `pdisplay` to propose an even nicer representation, where we see the different components of the `StateVector`:

In [ ]:
pcvl.pdisplay(sv2) # you can add nsimplify=False here too!

Finally, we can iterate manually through the different components of a state vector:

In [ ]:
for bs, alpha in sv2:
    # here we invoke manually the number simplification - easier to read
    print(alpha, pcvl.simple_complex(alpha)[1], bs)

We can combine states with different number of photons (`-2*sqrt(5)*I/5*|2,3>+sqrt(5)/5*|0,1>`) but we cannot combine states with different number of modes - it does not make sense:

In [ ]:
try:
    pcvl.BasicState("|0,1>")+pcvl.BasicState("|0,1,2>")
except:
    print("!!! Exception was generated: we cannot have superposed states with different number of modes !!!")

In [ ]:
# ok - we can now manipulate StateVector - let us define a function giving us the probability of having one or mode photons in a given mode
def photon_probability(sv, mode):
    r"""should return the probability that when we measure `sv`, we observe at least one photon in mode `mode`"""
    assert(mode < sv.m)
    probability = 0
    ### ENTER CODE HERE
    ### END CODE
    return probability

In [ ]:
# let us check this
assert(abs(photon_probability(pcvl.BasicState([0,1])+pcvl.BasicState([1,0]),0)-0.5)<1e-6)
assert(abs(photon_probability(pcvl.BasicState([0,1,0])+pcvl.BasicState([0,1,0]),0)-0)<1e-6)
assert(abs(photon_probability(sv2, 0)-0.8)<1e-6)

Now we can also use built-in `StateVector` methods providing sampling and measurement: 

In [ ]:
print(sv2)
c = Counter()
for s in sv2.samples(100): 
    c[s] += 1
print(c)

In [ ]:
sv_4 = pcvl.StateVector("|0,1,1>")-1j*pcvl.StateVector("|1,1,0>")
map_measure_sv_4 = sv_4.measure([1])
for s, (p, sv) in map_measure_sv_4.items(): 
    print(s, p, sv)

In [ ]:
map_measure_sv_4 = sv_4.measure([2])
for s, (p, sv) in map_measure_sv_4.items(): 
    print(s, p, sv)

## Modeling of a noisy photon source with a SVDistribution

A `SVDistribution` is a mixed state of `StateVector` - it is commonly used to model noisy photon source.

For that let us use the `pcvl.Source` object, it takes the following parameters:
* `emission_probability`: `float` = 1,
* `multiphoton_component`: `float` = 0,
* `indistinguishability`: `float` = 1,
* `losses`: `float` = 0
  

### Perfect Source

<img src="img/perfect_source.png">

In [ ]:
perfect_source = pcvl.Source(emission_probability=1, multiphoton_component=0, indistinguishability=1, losses=0)
pcvl.pdisplay(perfect_source.probability_distribution())

<div class="alert alert-info">
    In the perfect case, the source emits a sequence of `|1>`
</div>

### Real World Photon source

<img src="img/realworld_source.png">

In [ ]:
realworld_source = pcvl.Source(emission_probability=0.5, multiphoton_component=0.02, indistinguishability=0.95, losses=0.7)
pcvl.pdisplay(realworld_source.probability_distribution())
probability_1photon = 0
for sv, p in realworld_source.probability_distribution().items():
    probability_1photon += photon_probability(sv, 0)*p
print("probability of seing a photon in the output:", probability_1photon)

<div class="alert alert-warning">
    In the real world, with the parameters given to the source, 85% of the time, no photon is detected, 14.5% of the time, single indistinguishable photons are emitted (represented with <code>{_:0}</code>) - and the rest is a combination of g_2 and distinguishable photons
</div>

# LO-Components

The linear optical components are the elementary blocks which will act on our Fock states.

It's important to know all the possible components that can be found in Perceval and understand their effects.

In [ ]:
# the components we are interested in are in unitary_components or non_unitary_components
# see https://perceval.quandela.net/docs/components.html for the full list
import perceval.components.unitary_components as pcvl_comp

In [ ]:
# let us start with permutation - we define permutation by a permutation list
perm_1=pcvl_comp.PERM([2,0,1])
# name of the component
print(perm_1.name)
# the definition of the component
print(perm_1.describe())
# the unitary matrix
pcvl.pdisplay(perm_1.definition())
# and the visual representation
pcvl.pdisplay(perm_1)

In [ ]:
# let us define a function generating the permutation inverting first and last mode
def invert_first_last(m):
    ## ENTER CODE HERE
    ## END_CODE
    return perm

In [ ]:
perm_2 = invert_first_last(3)
print("the unitary:")
U_perm_2=perm_2.compute_unitary()
assert(U_perm_2[0,0]==0)
assert(U_perm_2[0,1]==0)
assert(U_perm_2[0,2]==1)
assert(U_perm_2[1,0]==0)
assert(U_perm_2[1,1]==1)
assert(U_perm_2[1,2]==0)
assert(U_perm_2[2,0]==1)
assert(U_perm_2[2,1]==0)
assert(U_perm_2[2,2]==0)
pcvl.pdisplay(U_perm_2)
print("the circuit:")
pcvl.pdisplay(perm_2)

In [ ]:
# let us do the same with beamsplitters !
bs=pcvl_comp.BS()
# name of the component
print(bs.name)
# the definition of the component
print(bs.describe())
# the unitary matrix
pcvl.pdisplay(bs.definition())
# the actual unitary matrix
pcvl.pdisplay(bs.compute_unitary())
# the visual representation
pcvl.pdisplay(bs)

In [ ]:
bs_rx = pcvl_comp.BS.Rx()  # By default a beam splitter follows the Rx gate convention (see documentation), so bs=BS() has the same matrix

# But other conventions exist too:
bs_h = pcvl_comp.BS.H() 
bs_ry = pcvl_comp.BS.Ry()

## Check the difference in the unitary definition:
print("BS.Rx() unitary matrix")
pcvl.pdisplay(bs_rx.definition())
print("BS.H() unitary matrix")
pcvl.pdisplay(bs_h.definition())
print("BS.Ry() unitary matrix")
pcvl.pdisplay(bs_ry.definition())
print("BS displays its convention as a small label")
pcvl.pdisplay(bs_ry)

In [ ]:
# You can ask for the symbolic matrix value of your component with the attribute U
my_ps = pcvl_comp.PS(phi=np.pi/8)
pcvl.pdisplay(my_ps.U)
# And for the numerical value with the method compute_unitary
pcvl.pdisplay(my_ps.compute_unitary())
print("")

# If you do it for a Beam-Splitter, you can see that by default theta=pi/2, and the phi's are 0
print("A default beam-splitter:")
pcvl.pdisplay(pcvl_comp.BS().compute_unitary())  #this is a balanced Beamsplitter
print("")

# To control the value of the parameters of a component, several choices are possible: 
#  - by setting a numerical value during the creation of the component
print("A Beam-Splitter with a numerical value for theta:")
bs_rx = pcvl_comp.BS.Rx(theta=10)
pcvl.pdisplay(bs_rx.U)
pcvl.pdisplay(bs_rx.compute_unitary())
print("")

In [ ]:
#  - by using the syntax pcvl.P to create a symbolic variable 
#    (note that you cannot compute the numerical value of your component anymore)
print("A Phase Shifter with a symbolic value for phi:")
ps = pcvl_comp.PS(phi=pcvl.P('\psi'))
pcvl.pdisplay(ps.U)
print("")

#  - you can still modify the value of a symbolic variable after its creation
#    This is not true for a numerical variable!
print("A beam-splitter with a symbolic variable...")
bs_rx = pcvl_comp.BS(theta=pcvl.P('a'))
pcvl.pdisplay(bs_rx.U)
bs_rx.assign({'a':10})
print("... set to a numerical value")
pcvl.pdisplay(bs_rx.compute_unitary())
print("")

In [ ]:
pcvl.pdisplay(bs_rx)

# LO-Circuits

From the LO-components, we can build a LO-circuit, i.e. a sequence of those components acting on our different modes.

## 1. Syntax

In [ ]:
circuit = pcvl.Circuit(3)  # Create a 3 mode circuit


circuit.add(0, pcvl_comp.BS())
circuit.add(0, pcvl_comp.PS(phi=np.pi/2)).add(1, pcvl_comp.PS(phi=pcvl.P('phi'))).add(1, pcvl_comp.BS())

# Equivalent syntax:
# circuit // BS() // PS(phi=np.pi/2) // (1, PS(phi=pcvl.P('phi'))) // (1, BS())

pcvl.pdisplay(circuit.U)
pcvl.pdisplay(circuit)

The syntax ``pcvl.P('phi')`` allows you to use parameters in the circuit, where you can assign a value or not. The behavior of the parameters of a circuit is similar to the case of the components.

For instance, you can use :

In [ ]:
params=circuit.get_parameters()
print(params) #list of the parameters

# the value is None, but we can change that with :

params[0].set_value(np.pi)
pcvl.pdisplay(circuit.compute_unitary(use_symbolic=False))
pcvl.pdisplay(circuit)

In [ ]:
# let us combine perm_2 with perm_2 to build identity
circuit = perm_2 // perm_2
pcvl.pdisplay(circuit.U)
pcvl.pdisplay(circuit)

## 2. Mach-Zehnder Interferometers

The beamsplitter's angle $\theta$ can also be defined as a parameter.

However, as the reflexivity depends on the mirror, it's hard to have adaptibility on the angle. 
Therefore, in practice, we use a [Mach-Zehnder Interferometer](https://en.wikipedia.org/wiki/Mach%E2%80%93Zehnder_interferometer). 

The beamsplitter with a parameterised $\theta$ is therefore implemented with a parameterised phase shifter $\phi$ between two fixed beamsplitters.

Let us try to reproduce the experiment from the presentation:
<img src="img/mzi.png">


In [ ]:
# Define the variable `mzi`to be a circuit implementing this schema. We want the phase-shifter on the top arm to be a parameter with 'phi' value
### ENTER CODE HERE
### END CODE
pcvl.pdisplay(mzi)

In [ ]:
import matplotlib.pyplot as plt

## We create a list of all different values for phi in the range [0,2*pi[
X = np.linspace(0, 2*np.pi, 1000)
Y = []
for phi in X:
    phase = mzi.get_parameters()[0]
    phase.set_value(phi)
    # we are interested in mzi.compute_unitary()[0,0] which is the probability of detecting a photon in first output mode
    Y.append(abs(mzi.compute_unitary()[0,0])**2)
   
plt.plot(X, Y)
plt.xlabel("phi")
plt.ylabel("R")
plt.show()

## 3. Universal Circuits

An operation on the modes of our circuit can also be expressed as a unitary.

For three modes, the unitary $U=\begin{pmatrix}
a_{1,1} & a_{1,2} & a_{1,3}\\
a_{2,1} & a_{2,2} & a_{2,3} \\ 
a_{3,1} & a_{3,2} & a_{3,3}
\end{pmatrix}$ performs the following operation on the Fock state basis:

$$\begin{array}{rcl}
|1,0,0\rangle &  \mapsto&  a_{1,1}|1,0,0\rangle + a_{1,2}|0,1,0\rangle + a_{1,3}|0,0,1\rangle\\
|0,1,0\rangle &  \mapsto&  a_{2,1}|1,0,0\rangle + a_{2,2}|0,1,0\rangle + a_{2,3}|0,0,1\rangle\\
|0,0,1\rangle &  \mapsto&  a_{3,1}|1,0,0\rangle + a_{3,2}|0,1,0\rangle + a_{3,3}|0,0,1\rangle
\end{array}$$

Since 1994, we know that any $U$ on the modes can be implemented as an LO-circuit [Reck's et al](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.73.58).

This decomposition can be done easily in Perceval using beamsplitters and phase-shifters as follows. 

In [ ]:
## From any unitary
n = 3
U = pcvl.Matrix.random_unitary(n)

circuit_u = pcvl.Circuit.decomposition(U, pcvl_comp.BS(theta=pcvl.P('theta'),phi_tr=pcvl.P('phi')), phase_shifter_fn=pcvl_comp.PS)

pcvl.pdisplay(circuit_u)

In [ ]:
print("The distance between the two unitaries is", np.linalg.norm(U-circuit_u.compute_unitary()))

In [ ]:
# do the same, but do not use a parameterable beamsplitter, but rather a mzi as defined here
mzi=pcvl.Circuit(2) // pcvl_comp.BS() // pcvl_comp.PS(pcvl.P("phi1")) //  pcvl_comp.BS()  //  pcvl_comp.PS(pcvl.P("phi2"))

### ENTER CODE HERE
### END CODE

pcvl.pdisplay(circuit_u_2)

In [ ]:
print("The distance between the two unitaries is", np.linalg.norm(U-circuit_u_2.compute_unitary()))

## 4. Black Box

To improve readibility, the circuit can be constructed in multiple steps which are then combined as black boxes. This will also help when we'll need generic operations.


In [ ]:
pre_MZI = (pcvl.Circuit(4, name="Bell State Prepar.")
           .add(0, pcvl_comp.BS())
           .add(2, pcvl_comp.BS())
           .add(1, pcvl_comp.PERM([1, 0])))

upper_MZI = (pcvl.Circuit(2, name="upper MZI")
             .add(0, pcvl_comp.PS(phi=pcvl.P('phi_0')))
             .add(0, pcvl_comp.BS())
             .add(0, pcvl_comp.PS(phi=pcvl.P('phi_2')))
             .add(0, pcvl_comp.BS()))

lower_MZI = (pcvl.Circuit(2, name="lower MZI")
             .add(0, pcvl_comp.PS(phi=pcvl.P('phi_1')))
             .add(0, pcvl_comp.BS())
             .add(0, pcvl_comp.PS(phi=pcvl.P('phi_3')))
             .add(0, pcvl_comp.BS()))

chip = (pcvl.Circuit(4)
              .add(0, pre_MZI)
              .add(0, upper_MZI, merge=False)
              .add(2, lower_MZI, merge=False))

pcvl.pdisplay(chip)

In [ ]:
## You can still display the inside of black boxes with:
pcvl.pdisplay(chip, recursive=True)

# Simulation

Up to this point, we have focused on creating circuits.
It's time to learn how to sample from them or describe their output distribution, on many different inputs.

## 1. Computing probabilities

For this part, we will take the [Hong-Ou-Mandel](https://en.wikipedia.org/wiki/Hong%E2%80%93Ou%E2%80%93Mandel_effect) experience as an example.

It's one of the simplest experiments and yet it is very useful.

Making two indistinguishable photons, one in each mode, enter one balanced beamsplitter $BS=\frac{1}{\sqrt{2}} \left[\begin{matrix}1 & 1\\1& -1\end{matrix}\right]$, we expect the outcome to be:

$$|1,1\rangle \mapsto \frac{|2,0\rangle - |0,2\rangle}{\sqrt{2}}  $$

We will show how to verify this in the next steps using the Naive backend to recover the full probability distribution.

In [ ]:
circuit = pcvl.Circuit(2) // pcvl_comp.BS.H()
pcvl.pdisplay(circuit)

### Getting probability distribution with computing backend

In [ ]:
backend = pcvl.BackendFactory.get_backend("Naive")
backend.set_circuit(circuit)
backend.set_input_state(pcvl.BasicState([1,1]))
print(backend.prob_amplitude(pcvl.BasicState([2,0])))  #note that it's the amplitude !
print(backend.prob_amplitude(pcvl.BasicState([0,2])))
print(backend.probability(pcvl.BasicState([0,2])))

In [ ]:



## We can also use the Analyser module to compute a table of probabilities
## The Analyser uses a Processor to work with. A Processor aims at simulating a photonic source plugged into a circuit
## with a given backend.
## The main syntax is :
## >>> p = pcvl.Processor(backend_name, circuit, source)
p = pcvl.Processor("Naive", circuit)
analyzer = pcvl.algorithm.Analyzer(p, [pcvl.BasicState([1,1])], '*')
pcvl.pdisplay(analyzer)

In [ ]:
# Let us do that on a random unitary

randU=pcvl.Unitary(pcvl.Matrix.random_unitary(3))   
input=pcvl.BasicState([1,1,0])

p=pcvl.Processor("SLOS", randU)  #We can put in the processor a pcvl.Unitary instead of a circuit ! We don't need to use pcvl.decomposition
analyzer= pcvl.algorithm.Analyzer(p, [input], '*')
pcvl.pdisplay(analyzer)

### 2. Sampling

Although it's crucial to compute the output distribution, it's not what we can expect from a photonic chip. Indeed, realistically, we only can obtain a single sample from the distribution each time we run the circuit. This can be done using the backend SLOS.


In [ ]:
p = pcvl.Processor("SLOS", pcvl_comp.BS())
p.with_input(pcvl.BasicState([1,1]))

# The sampler holds 'probs', 'sample_count' and 'samples' calls. You can use the one that fits your needs!
sampler = pcvl.algorithm.Sampler(p)  

# A sampler call will return a Python dictionary containing sampling results, and two performance scores
# sample_count = sampler.sample_count(1000)
# sample_count contains {'results': <actual count>, 'physical_perf': float [0.0 - 1.0], 'logical_perf': float [0.0 - 1.0]}
sample_count = sampler.sample_count(1000)
print(sample_count['results'])


In [ ]:
# Let us do the same for the random matrix with 2 photon inputs on mode 0 and 1
# Define processor and input - you can change backend - see https://perceval.quandela.net/docs/backends.html

### ENTER CODE HERE
### END CODE

sampler = pcvl.algorithm.Sampler(p)  
sample_count = sampler.sample_count(1000)
print(sample_count['results'])

## Question: how many states do we have for 3 modes and 2 photons?
## There are 6 different states


## Question : how many states do we have for m modes and n photons? 
## There are m+n-1 choose n different states. Cf Bar and Star problems.

Note : to approximate with decent precision a distribution over $M$ different states, we would need $M^2$ samples. This can be shown by [Hoeffding's inequality](https://en.wikipedia.org/wiki/Hoeffding%27s_inequality). 

### 3. Sampling with real world source

In the precedent cells, we have simulated a quantum processor using a perfect source, `Processor` initialization takes a third parameter which is a `Source` object as defined above

In [ ]:
# Run the sampling again on the random unitary, but setting a real world source - compare the results

### ENTER CODE HERE
### END CODE
p.min_detected_photons_filter(0)  # Do not filter out any output state

sampler_realworld = pcvl.algorithm.Sampler(p)  
sample_count_realworld = sampler.sample_count(1000)
print(sample_count['results'])

## Encoding Qubits

### 1. Path encoding

To perform quantum computations using photons, we need an encoding: a correspondance between our Fock states and our qubit states.

We therefore want to associate each qubit state with one of our Fock states.

One natural way to encode qubits is the path encoding.
A qubit is a two-level quantum state, so we will use two spatial modes to encode it: this is the dual-rail or path encoding.

The logical qubit state $|0\rangle_L$ will correspond to a photon in the upper mode, as in the Fock state $|1,0\rangle$, while $|1\rangle_L$ will be encoded as $|0,1\rangle$.


We can extend this to multiple qubits by having twice as many modes as there are qubits. For example the $3$-qubit state $\frac{1}{\sqrt{2}}(|000\rangle_L+|111\rangle_L)$ can be encoded with $3$ photons and $3\times 2=6$ modes :
$\frac{1}{\sqrt{2}}(|1,0,1,0,1,0\rangle+|0,1,0,1,0,1\rangle)$

### 2. Single-qubit gates

Using the dual-rail enconding, single-qubit gates only deal with one photon and are straightforward. Can you give the LO-circuits for the gates below?

$$X=\left[\begin{matrix}0 & 1\\1& 0\end{matrix}\right]$$
$$Y=\left[\begin{matrix}0 & -i\\i& 0\end{matrix}\right]$$
$$Z=\left[\begin{matrix}1 & 0\\0& -1\end{matrix}\right]$$
$$H=\frac{1}{\sqrt{2}} \left[\begin{matrix}1 & 1\\1& -1\end{matrix}\right]$$

$$R_X=\left[\begin{matrix}\cos{\left(\frac{\theta}{2} \right)} & -i \sin{\left(\frac{\theta}{2} \right)}\\-i \sin{\left(\frac{\theta}{2} \right)} & \cos{\left(\frac{\theta}{2} \right)}\end{matrix}\right]$$

$$R_Y=\left[\begin{matrix}\cos{\left(\frac{\theta}{2} \right)} & - \sin{\left(\frac{\theta}{2} \right)}\\ \sin{\left(\frac{\theta}{2} \right)} & \cos{\left(\frac{\theta}{2} \right)}\end{matrix}\right]$$

$$R_Z=\left[\begin{matrix}e^{-i\frac{\theta}{2}} & 0 \\ 0 & e^{i\frac{\theta}{2}}\end{matrix}\right]$$

In [ ]:
## find the LO-circuits for each gate and compare unitaries

### ENTER CODE HERE
### END CODE

### 3. Two-qubit gates

On the other hand, in dual-rail encoding, it can be shown that two-qubit gates can't be deterministic, and have a probability to fail.

There are two ways to detect that failure:

- We can use additional photons called ancillas, which we can measure independently from the main circuit photons. Depending on the state obtained on the ancilla, we know whether the gate has succeeded or not on the main qubits. Those gates will be called heralded.
- We can also directly measure the main circuit qubits, and depending on the result, assess whether the gate has succeeded or not. Those gates will be called postselected.

The CNOT gate acts on two qubits, a control and a target, and flips the value of the target if the control qubit is in state $|1\rangle_L$. In the following two exercices, we will see the two types of CNOT gates: 
- the postselected CNOT of [Ralph et al.](https://arxiv.org/abs/quant-ph/0112088)
- the heralded CNOT of the [KLM protocol](https://arxiv.org/abs/quant-ph/0006088)

In [ ]:
## We introduce the component catalog. It contains both CNOT gates.
from perceval.components import catalog
print(catalog.list())

In [ ]:
## Ralph's et al. CNot

print(catalog['postprocessed cnot'].doc)
ralph_cnot = catalog['postprocessed cnot'].build_processor()
## You can set its input state with a LogicalState
ralph_cnot.with_input(pcvl.LogicalState([1, 0]))

pcvl.pdisplay(ralph_cnot, recursive=True, render_size=1.25)

In [ ]:
## You can sample some output states
h_cnot = catalog['klm cnot'].build_processor()
cnot_sampler = pcvl.algorithm.Sampler(h_cnot)
h_cnot.with_input(pcvl.LogicalState([0, 0]))

samples = cnot_sampler.samples(10)
print(samples['results'])
print("Some output states were not selected because of heralds and post-processing => you can check the logical performance")
print("Logical performance =", samples['logical_perf'])

In [ ]:
states = {
    pcvl.BasicState([1, 0, 1, 0]): "00",
    pcvl.BasicState([1, 0, 0, 1]): "01",
    pcvl.BasicState([0, 1, 1, 0]): "10",
    pcvl.BasicState([0, 1, 0, 1]): "11"
}

ca = pcvl.algorithm.Analyzer(h_cnot, states)
ca.compute(expected={"00": "00", "01": "01", "10": "11", "11": "10"})
pcvl.pdisplay(ca)
print(f"performance = {pcvl.simple_float(ca.performance)[1]}, fidelity = {ca.fidelity*100}%")

In [ ]:
from perceval.algorithm import ProcessTomography
from perceval.algorithm.tomography import is_physical
h_cnot = catalog['klm cnot'].build_processor()
qpt = ProcessTomography(operator_processor=h_cnot)
chi_op = qpt.chi_matrix()  # computing the chi matrix
pcvl.pdisplay(qpt, render_size=(15,30))  # visualization of the same chi
